<a href="https://colab.research.google.com/github/joydeep049/Pytorch_Practice/blob/main/PyTorch_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as t
t.__version__

'2.8.0+cu126'

## Tensors

*   A multi-dimensional matrix containing elements of a single data type.
*   Similar to Numpy Arrays, but work better in GPU.
*   default data type -> float32  
*   More suitable for deep learning than numpy array





In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
BATCH_SIZE = 64

In [3]:
 training_data = datasets.FashionMNIST(
     root = "data",
     train = True,
     download = True,
     transform = ToTensor(),
 )

 test_data = datasets.FashionMNIST(
     root = "data",
     train = False,
     download = True,
     transform = ToTensor(),
 )

100%|██████████| 26.4M/26.4M [00:02<00:00, 9.89MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 201kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.67MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 3.42MB/s]


In [4]:
train_dataloader = DataLoader(training_data, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size = BATCH_SIZE)

# for X,y in test_dataloader:
#   print(f"Shape of X is {X.shape}")
#   print(f"Shape of Y is {y.shape}")

In [5]:
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu"

print(f"Using device {device}")

# model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.conv_layers = nn.Sequential(
      nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding  = 1), # (32,28,28)
      nn.ReLU(),
      nn.MaxPool2d(2,2), #(32,14,14)

      nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1), #(64,14,14)
      nn.ReLU(),
      nn.MaxPool2d(2,2), # (64,7,7)
    )
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(64*7*7, 128),
        nn.ReLU(),
        nn.Linear(128,10),
    )

  def forward(self,x):
    x = self.conv_layers(x)

    x = self.flatten(x)

    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using device 0
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss() # applies softmax as well
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)



In [7]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1)* len(X)
      print(f"Loss: {loss:>7f}, [{current:>5d},{size:>5d}]")

In [8]:
def test(dataloader,model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0

  with torch.no_grad():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 15
for t in range(epochs):
  print(f"Epoch {t+1}")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader,model, loss_fn)

print("Training Completed")

Epoch 1
Loss: 2.292893, [   64,60000]
Loss: 2.291082, [ 6464,60000]
Loss: 2.281209, [12864,60000]
Loss: 2.291517, [19264,60000]
Loss: 2.284773, [25664,60000]
Loss: 2.267643, [32064,60000]
Loss: 2.286208, [38464,60000]
Loss: 2.259909, [44864,60000]
Loss: 2.258695, [51264,60000]
Loss: 2.260773, [57664,60000]
Test Error: 
 Accuracy: 34.4%, Avg loss: 2.245662 

Epoch 2
Loss: 2.240860, [   64,60000]
Loss: 2.234904, [ 6464,60000]
Loss: 2.209342, [12864,60000]
Loss: 2.219578, [19264,60000]
Loss: 2.179662, [25664,60000]
Loss: 2.165359, [32064,60000]
Loss: 2.160740, [38464,60000]
Loss: 2.108382, [44864,60000]
Loss: 2.084281, [51264,60000]
Loss: 2.032320, [57664,60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 2.002615 

Epoch 3
Loss: 2.017642, [   64,60000]
Loss: 1.955471, [ 6464,60000]
Loss: 1.821006, [12864,60000]
Loss: 1.791782, [19264,60000]
Loss: 1.583624, [25664,60000]
Loss: 1.547938, [32064,60000]
Loss: 1.445332, [38464,60000]
Loss: 1.319040, [44864,60000]
Loss: 1.303103, [51264,60000]
Lo

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.unsqueeze(0).to(device) # (1,28,28) -> (1,1,28,28), add a new dim at index 0, cause model expects a batch dimension as well.
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
